# Example Usage of Preprocessors and DocumentProcessorPipe

## Create some Dummy Documents

In [85]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path="data/merkblatt-fuer-arbeitslose_ba036520.pdf")
pages = loader.load_and_split()
examples = pages[20:25]

In [86]:
for i, example in enumerate(examples):
    print(f"Page {i}")
    print(50*"#")
    print(example.page_content)
    print(50*"#" + "\n")

Page 0
##################################################
2. Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?
22Welche konkreten Aktivitäten Sie im Rahmen der 
Arbeit  suche unternehmen bzw. wie Sie Ihre Eigen -
bemühungen nachweisen müssen, entnehmen Sie 
 Ihrer Eingliederungsvereinbarung bzw. der schriftlichen 
Festsetzung Ihrer Eigenbemühungen. Erbringen Sie 
die Pflichten im Zusammenhang mit den Eigenbemü ­
hungen nicht, nicht rechtzeitig oder nicht vollständig, 
tritt eine Sperrzeit ein (siehe hierzu » Abschnitt 6) .
Wollen Sie die Pflichten aus der Eingliederungsverein ­
barung nicht erfüllen bzw. keine Eigenbemühungen un ­
ternehmen, haben Sie keinen Leistungsanspruch bzw. 
kann Ihr Leistungsanspruch – gegebenenfalls rückwir ­
kend – entfallen.
Falls Sie an einer Beschäftigung im Ausland 
(EU, EWR oder der Schweiz)  interessiert sind und 
dort eine Beschäftigung suchen wollen, können Sie das 
deutsche Arbeitslosengeld für einen begrenzten Zeit ­
raum weiterhin im Ausland bezi

## Use a single Preprocessor - For one document

- If you're using a preprocessor, it will overwrite the document's page_content inplace!
- You can use the preprocessor on single string too

In [87]:
from textprepper.removers import RemoveNewLines
from textprepper.modifiers import UpperText

In [88]:
original = examples[0]
example = original.copy()
print_index = 500

In [89]:
print(original.page_content[:print_index])

2. Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?
22Welche konkreten Aktivitäten Sie im Rahmen der 
Arbeit  suche unternehmen bzw. wie Sie Ihre Eigen -
bemühungen nachweisen müssen, entnehmen Sie 
 Ihrer Eingliederungsvereinbarung bzw. der schriftlichen 
Festsetzung Ihrer Eigenbemühungen. Erbringen Sie 
die Pflichten im Zusammenhang mit den Eigenbemü ­
hungen nicht, nicht rechtzeitig oder nicht vollständig, 
tritt eine Sperrzeit ein (siehe hierzu » Abschnitt 6) .
Wollen Sie die Pflichten au


In [90]:
# upper casing the document page_content
preprocessor = UpperText()
print(preprocessor(example).page_content[:print_index])

2. WAS MÜSSEN SIE TUN, WENN ARBEITSLOSIGKEIT EINTRITT?
22WELCHE KONKRETEN AKTIVITÄTEN SIE IM RAHMEN DER 
ARBEIT  SUCHE UNTERNEHMEN BZW. WIE SIE IHRE EIGEN -
BEMÜHUNGEN NACHWEISEN MÜSSEN, ENTNEHMEN SIE 
 IHRER EINGLIEDERUNGSVEREINBARUNG BZW. DER SCHRIFTLICHEN 
FESTSETZUNG IHRER EIGENBEMÜHUNGEN. ERBRINGEN SIE 
DIE PFLICHTEN IM ZUSAMMENHANG MIT DEN EIGENBEMÜ ­
HUNGEN NICHT, NICHT RECHTZEITIG ODER NICHT VOLLSTÄNDIG, 
TRITT EINE SPERRZEIT EIN (SIEHE HIERZU » ABSCHNITT 6) .
WOLLEN SIE DIE PFLICHTEN AU


In [91]:
# Removing all single newlines of the Document "\n"
preprocessor = RemoveNewLines(count=1)
print(preprocessor(example).page_content[:print_index])

2. WAS MÜSSEN SIE TUN, WENN ARBEITSLOSIGKEIT EINTRITT?22WELCHE KONKRETEN AKTIVITÄTEN SIE IM RAHMEN DER ARBEIT  SUCHE UNTERNEHMEN BZW. WIE SIE IHRE EIGEN -BEMÜHUNGEN NACHWEISEN MÜSSEN, ENTNEHMEN SIE  IHRER EINGLIEDERUNGSVEREINBARUNG BZW. DER SCHRIFTLICHEN FESTSETZUNG IHRER EIGENBEMÜHUNGEN. ERBRINGEN SIE DIE PFLICHTEN IM ZUSAMMENHANG MIT DEN EIGENBEMÜ ­HUNGEN NICHT, NICHT RECHTZEITIG ODER NICHT VOLLSTÄNDIG, TRITT EINE SPERRZEIT EIN (SIEHE HIERZU » ABSCHNITT 6) .WOLLEN SIE DIE PFLICHTEN AUS DER EIN


## Use the DocumentPipe with mutiple Preprocessors for a single document

- if you know torchvision's transform/compose logic, this should look familiar to you
- You can pass a list of preprocessors to the `DocumentPreprocessorPipe` class
- By calling the class instance on a document, it will apply each preprocessor sequentially
- you can use the `.from_documents()` method of the pipe class, which will loops over all passed documents

In [92]:
from textprepper import DocumentPreprocessorPipe

from textprepper.removers import RemoveNewLines, RemovePunctuation
from textprepper.modifiers import UpperText, LowerText

In [93]:
example_pipe = original.copy()
preprocessors = [
                 RemoveNewLines(count=1), # removes all single newlines (ONLY)
                 UpperText(), # you saw before what upper does
                 LowerText(), # -> This will lower all chars
                 RemovePunctuation() # Removes all punctuations
                 ]
doc_pipe = DocumentPreprocessorPipe(preprocessors)

In [94]:
print(doc_pipe(example_pipe).page_content[:print_index])

2 was müssen sie tun wenn arbeitslosigkeit eintritt22welche konkreten aktivitäten sie im rahmen der arbeit  suche unternehmen bzw wie sie ihre eigen bemühungen nachweisen müssen entnehmen sie  ihrer eingliederungsvereinbarung bzw der schriftlichen festsetzung ihrer eigenbemühungen erbringen sie die pflichten im zusammenhang mit den eigenbemü ­hungen nicht nicht rechtzeitig oder nicht vollständig tritt eine sperrzeit ein siehe hierzu » abschnitt 6 wollen sie die pflichten aus der eingliederungsve


In [95]:
pipe_examples = [doc.copy() for doc in examples]

In [96]:
results = doc_pipe.from_documents(pipe_examples)

In [97]:
results

[Document(page_content='2 was müssen sie tun wenn arbeitslosigkeit eintritt22welche konkreten aktivitäten sie im rahmen der arbeit  suche unternehmen bzw wie sie ihre eigen bemühungen nachweisen müssen entnehmen sie  ihrer eingliederungsvereinbarung bzw der schriftlichen festsetzung ihrer eigenbemühungen erbringen sie die pflichten im zusammenhang mit den eigenbemü \xadhungen nicht nicht rechtzeitig oder nicht vollständig tritt eine sperrzeit ein siehe hierzu » abschnitt 6 wollen sie die pflichten aus der eingliederungsverein \xadbarung nicht erfüllen bzw keine eigenbemühungen un \xadternehmen haben sie keinen leistungsanspruch bzw kann ihr leistungsanspruch – gegebenenfalls rückwir \xadkend – entfallenfalls sie an einer beschäftigung im ausland eu ewr oder der schweiz  interessiert sind und dort eine beschäftigung suchen wollen können sie das deutsche arbeitslosengeld für einen begrenzten zeit \xadraum weiterhin im ausland beziehen bitte informieren sie sich ggf vor ihrer ausreise  be

# Create a custom Preprocessor - Example Remove All Numbers

There are some weird numbers in the documents, so let's create a custom preprocessor which removes all numbers

- The preprocessor class has one abstract method which we need to define `process_text()`
  - this will get the string of the document, performs the string manipulation and needs to return the string back
  - optional: we can define the `add_metadata()` if we want to edit the metadata of the document too
- The preprocessor is build on pydantic base model

In [98]:
from textprepper import Preprocessor
from typing import Dict
from langchain_core.documents import Document


In [99]:
class RemoveAllNumbers(Preprocessor):
    def process_text(self, text: str, *args, **kwargs) -> str:
        return ''.join([char for char in text if not char.isdigit()])
    
    def add_metadata(self, doc: Document, **kwargs) -> Dict:
        return {"removed_all_numbers": True}

In [103]:
print(original.page_content[:print_index])

2. Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?
22Welche konkreten Aktivitäten Sie im Rahmen der 
Arbeit  suche unternehmen bzw. wie Sie Ihre Eigen -
bemühungen nachweisen müssen, entnehmen Sie 
 Ihrer Eingliederungsvereinbarung bzw. der schriftlichen 
Festsetzung Ihrer Eigenbemühungen. Erbringen Sie 
die Pflichten im Zusammenhang mit den Eigenbemü ­
hungen nicht, nicht rechtzeitig oder nicht vollständig, 
tritt eine Sperrzeit ein (siehe hierzu » Abschnitt 6) .
Wollen Sie die Pflichten au


In [104]:
remove_numbers = RemoveAllNumbers()
copy_doc = original.copy()

print(remove_numbers(copy_doc).page_content[:print_index])
print(copy_doc.metadata)

. Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?
Welche konkreten Aktivitäten Sie im Rahmen der 
Arbeit  suche unternehmen bzw. wie Sie Ihre Eigen -
bemühungen nachweisen müssen, entnehmen Sie 
 Ihrer Eingliederungsvereinbarung bzw. der schriftlichen 
Festsetzung Ihrer Eigenbemühungen. Erbringen Sie 
die Pflichten im Zusammenhang mit den Eigenbemü ­
hungen nicht, nicht rechtzeitig oder nicht vollständig, 
tritt eine Sperrzeit ein (siehe hierzu » Abschnitt ) .
Wollen Sie die Pflichten aus de
{'source': 'data/merkblatt-fuer-arbeitslose_ba036520.pdf', 'page': 21, 'removed_all_numbers': True}
